In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

data = pd.read_excel('your_file_name.xlsx')

#进行数据清洗：先把前面的人民币符号去掉便于进行数据可视化！
data['mean-price'] = data['mean-price'].replace('￥','')
#再因为人均消费栏存在缺失值，所以进行相邻值填充
data['mean-price'] = data['mean-price'].fillna(method='bfill') #这里选择的是相邻值填充的后值bfill法，相对合理
data['mean-price'] = data['mean-price'].astype(int)
#print(data['mean-price'][168:172])

#重新保存到原始表格中生成新的数据表
data.to_excel('your_file_name.xlsx',sheet_name='大众点评成都咖啡店650家信息')

In [ ]:
data.describe()

In [ ]:
#看哪个区的咖啡店数量最多
plt.rcParams['font.sans-serif']=['SimHei'] 
plt.figure(figsize=(12,10)) 
data['area'] .value_counts().plot.bar() 
plt.show()

In [ ]:
# -*- coding: utf-8 -*-
#可视化区域地图，生成html文件，在浏览器里面查看
from pyecharts.charts import Map
from pyecharts import options as opts

# 成都市行政区划分
chengdu = {'武侯区': 191, '锦江区': 155, '青羊区': 75,
           '成华区': 58, '金牛区': 38, '双流区': 36,
           '郫都区': 19, '温江区': 18, '龙泉驿区': 14,
           '新都区': 14, '都江堰市': 10, '金堂县': 5,
           '青白江区': 5, '大邑县': 3, '简阳市': 3,
           '蒲江县': 3, '彭州市': 2, '邛崃市': 1
          }

map = Map()
map.set_global_opts(
    title_opts=opts.TitleOpts(title="大众点评成都前650家咖啡店地区分布图"),
    #datazoom_opts = opts.DataZoomOpts(is_zoom_lock=False)  #区域缩放配置
    visualmap_opts=opts.VisualMapOpts(max_=3600, is_piecewise=True,
                                      pieces=[
                                        {"max": 200, "min": 181, "label": ">180", "color": "#E74C3C"},
                                        {"max": 180, "min": 100, "label": "180-100", "color": "#5DADE2"},
                                        {"max": 99, "min": 51, "label": "99-51", "color": "#F5B041"},
                                        {"max": 50, "min": 21, "label": "50-21", "color": "#F4D03F"},
                                        {"max": 20, "min": 11, "label": "20-11", "color": "#F5B7B1"},
                                        {"max": 10, "min": 0, "label": "10-0", "color": "#FFFFFF"},
                                        ], )  #最大数据范围，分段
    )
map.add("成都咖啡店地址分布图", data_pair=chengdu.items(), maptype="成都", is_roam=True, zoom=1.2) #最后这个zoom是当前视角的缩放比例
map.render('大众点评成都650咖啡店分布图.html')

#如果不想在浏览器查看，可以直接在jupyter notebook里面输出，使用如下命令即可：
#map.render_notebook()

In [ ]:
#对各指标进行一个整体的直方图可视化：先是人均消费
plt.figure(figsize=(12,10))
sns.distplot(data['mean-price'],kde=True,rug=True,color='r')
plt.show()

In [ ]:
#现在我们去除那些大于100的异常咖啡厅数值，重新来算人均消费的平均值
data1 =data.copy()
#data1
data1 = data1[data1['mean-price']<=100] #筛选某一列里面的部分数据的方法
data1.sort_values(by='mean-price',ascending=False)
data1.describe()

In [ ]:
#再次查看人均消费的价格分布
plt.figure(figsize=(12,10))
sns.distplot(data1['mean-price'],kde=True,rug=True,color='b')
plt.show()

In [ ]:
#对点评数进行一个直方图显示，可以看出500以下的评论数最多
plt.figure(figsize=(15,10))
sns.distplot(data['review-num'],kde=True,rug=True,color='g')
plt.show()

In [ ]:
data.sort_values(by='review-num',ascending=False)[:10]

In [ ]:
#统计推荐菜里面重复次数最高的饮品
#但是这个是整体统计的，并不是单个推荐菜的次数
data['recommend'].value_counts()

In [ ]:
def sum_recommend():
    sum_rec = data['recommend'][0]
    try:
        for i in range(650):
            sum_rec = sum_rec + ',' + data['recommend'][i+1]
    except TypeError:
        pass
    return sum_rec

if __name__ == '__main__':
    rec = sum_recommend()
    rec = rec.split(',') #再把字符串转换成列表方便统计数量
    rec_count = pd.value_counts(rec) #然后统计哪个饮品出现次数最多
    #print(rec_count)
    plt.rcParams['font.sans-serif']=['SimHei'] 
    plt.figure(figsize=(12,10)) 
    rec_count[:10].plot.barh(color='c') 
    plt.show()

In [ ]:
#下面进行词云图的生成
#-*- coding:utf-8 -*-
from wordcloud import WordCloud,ImageColorGenerator
#词云库
import PIL.Image as Image
import jieba
import jieba.analyse
import os
from os import path

rec1 = rec
jieba = ' '.join(rec1)
#jieba分词，cut_all=False 表示采用分词的精确模式
#jieba = ' '.join(jieba.cut(rec1,cut_all = 'False'))
#采用TF-IDF算法来分词
#jieba = ' '.join(jieba.analyse.extract_tags(rec1, topK=20, withWeight=False, allowPOS=()))

cp = np.array(Image.open('卡布.jpg'))

wordcloud = WordCloud(background_color='white', max_words=2000, mask=cp,
                         max_font_size=40, random_state=42, font_path='./SimHei.ttf').generate(jieba)

#基于彩色的图片生成相应的色彩
image_colors = ImageColorGenerator(cp)

plt.imshow(wordcloud.recolor(color_func=image_colors))
plt.axis('off')
plt.show()

wordcloud.to_file(os.path.join('coffee_k.jpg'))

In [ ]:
#接着也对口味、环境、服务这几个方面也进行一个直方图的显示
plt.figure(figsize=(15,10))
sns.distplot(data['taste-num'],kde=True,rug=True,color='b')
plt.show()

In [ ]:
#环境分
plt.figure(figsize=(15,10))
sns.distplot(data['envir-num'],kde=True,rug=True,color='y')
plt.show()

In [ ]:
#服务分
plt.figure(figsize=(15,10))
sns.distplot(data['service-num'],kde=True,rug=True,color='m')
plt.show()

In [ ]:
data4 = pd.pivot_table(data, index=['name'], values=['taste-num','envir-num','service-num']) #生成评分的数据透视表
data4

In [ ]:
#综合口味、服务、环境三项的得分后平均分排列前10的咖啡店
data4.sort_values(by=['taste-num','envir-num','service-num'],ascending=False)[:10] 

In [ ]:
#650家店里面评分最差的10家咖啡店
data4.sort_values(by=['taste-num','envir-num','service-num'])[:10] 

In [ ]:
#下面进行多变量分析
#这个有点牛逼，一行代码直接甩出数据集里面的任意两个变量之间的关系
sns.pairplot(data)

In [ ]:
#如果想对分类变量进行着色，让这个矩阵更易理解,也只需要一行代码:这里采用area分类变量
plt.rcParams['font.sans-serif']=['SimHei'] 
sns.pairplot(data, hue='area')

In [ ]:
#加入趋势线
sns.pairplot(data, kind='reg')